In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
from libs import *
from os.path import join
import torch
from tqdm import tqdm
dataSetPath = "dataSet/"

下载数据

In [3]:
downLoadData = DownloadData(Token=token, dataSetPath=dataSetPath)

downLoadData.setStockList("../StockPriceForecast/stockList.csv")
downLoadData.storeData()

DownLoad Information : 
    Download Dates :  3022   Days
    Stock quantity :  1440   Stocks
    Parameters :      11   Parameters


100%|██████████| 3022/3022 [41:11<00:00,  1.22it/s] 


处理数据

In [ ]:
dataProcess = DataProcess(dataSetPath=dataSetPath)
dataProcess.loadData()
dataProcess.dropDataFrame(labels=["ts_code", "trade_date"], dim="Parameter")
dataProcess.copyDataFrame(label="change", dim="Parameter")
dataProcess.minMaxNormal()
dataProcess.storeData()

加载数据

In [2]:
dataPath = join(dataSetPath, "data/marketData/data.nc")
data_len = DataLength(dataPath)
print(data_len)

2994


In [4]:
input_days = 360
predict_days = 30
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[0, data_len - input_days - test_predict_len])
print(train_dataSet.data.shape)
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len])
print(validation_dataSet.data.shape)
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len])
print(test_dataSet.data.shape)
#      |-------------train data 2214-------------|-30-|
#      |                                |---validation data 720---|-----389-----|
#Data: --------------------------------------------------------------------------
#                                       |-----389-----|------test data 720------|

(2274, 1440, 10)
(720, 1440, 10)
(720, 1440, 10)


模型训练

In [ ]:
module = modules.ResNetLSTM()
module.cuda()
output = 0

In [ ]:
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
torch.cuda.empty_cache()
torch.set_grad_enabled(False)
for train_data, target_data in tqdm(dataLoader):
    train_data = train_data.cuda()
    train_data = train_data.reshape(360, 1, 120, 120)
    output = module(train_data)

    torch.cuda.empty_cache()
    break

In [ ]:
print(output.shape)

获取预测数据

In [ ]:
import torch


数据可视化

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

数据分析